# Data Structures on the system

### Prerequisites



Run these commands in the terminal pane of your VSCode session in CodeSpaces.
Not needed if running this notebook in Binder.

In [ ]:
sudo apt-get update
sudo apt-get install -y wamerican-insane

In [2]:
commands="tree find ls cat head tail wc fgrep cut seq shuf sed awk"
<<< ${commands} tr ' ' '\n' | wc -l
which ${commands} | wc -l

13
13


The system has a number of data structures that it uses.  The two that we will be exploring are these:
- the file system
- a file

And like any other data structure, we can perform the four CRUD operations on them: Create, Read, Update, Delete.

### The file system

The file system is where all data is stored and organized.  The structure is that of a hierarchy or tree with branches eminating from a "root" and from "nodes" ( aka branch points ) and terminating in "leaves".  


### A file

A file is just a series of bytes of a finite length stored on the file system.

## The file system

One command that we can use to display the file system hierarch is `tree`.

For example:

In [1]:
tree -F /etc/apt/

/etc/apt/
├── apt.conf.d/
│   ├── 01autoremove
│   ├── 01autoremove-kernels
│   ├── 01-vendor-ubuntu
│   ├── 70debconf
│   ├── docker-autoremove-suggests
│   ├── docker-clean
│   ├── docker-gzip-indexes
│   └── docker-no-languages
├── auth.conf.d/
├── preferences.d/
├── sources.list
├── sources.list.d/
└── trusted.gpg.d/
    ├── ubuntu-keyring-2012-archive.gpg
    ├── ubuntu-keyring-2012-cdimage.gpg
    └── ubuntu-keyring-2018-archive.gpg

5 directories, 12 files


In [3]:
find /etc/apt


/etc/apt
/etc/apt/sources.list.d
/etc/apt/sources.list
/etc/apt/auth.conf.d
/etc/apt/apt.conf.d
/etc/apt/apt.conf.d/docker-clean
/etc/apt/apt.conf.d/01autoremove
/etc/apt/apt.conf.d/docker-no-languages
/etc/apt/apt.conf.d/docker-autoremove-suggests
/etc/apt/apt.conf.d/01autoremove-kernels
/etc/apt/apt.conf.d/01-vendor-ubuntu
/etc/apt/apt.conf.d/70debconf
/etc/apt/apt.conf.d/docker-gzip-indexes
/etc/apt/preferences.d
/etc/apt/trusted.gpg.d
/etc/apt/trusted.gpg.d/ubuntu-keyring-2012-archive.gpg
/etc/apt/trusted.gpg.d/ubuntu-keyring-2018-archive.gpg
/etc/apt/trusted.gpg.d/ubuntu-keyring-2012-cdimage.gpg


( We will explain the details of using `tree` and other commands later.  For now, you can just run the commands to see what ouput they generate. )

Here we see a portion of the file system hierarchy.  The tree is made up of only two elements: directories ( aka folders ) and files.  Directories can contain other directories ( aka subdirectories ) and files.  In this example, directories are denoted by ending with a '/' symbol.  Here we have four directories: home, jovyan, Initial.Data.Analysis, and LinuxTerminal.  The home folder has a single child, jovyan, which itself has a number of children, including two subdirectories, Initial.Data.Analysis and LinuxTerminal.  The LinuxTerminal directory contains no subdirectories and has only a single file, Linux.Terminal.Setup.ipynb.

Notice that the home directory is prefixed with a slash '/'.  This is the symbol for the root of the tree or the "root" directory. The slash is also the symbol used to delineate directories from subdirectories.  For example, we can list the files and folders in a specific directory by using the `ls` command:

In [2]:
ls -F /home/jovyan/

apt.txt
environment.yml
hello.jl
hello.R
Initial.Data.Analysis/
LICENSE
LinuxTerminal/
Manifest.toml
postBuild*
Project.toml
README.md
runtime.txt
start*


A few items to note:
1. all subdirectories end with a slash '/'. 
1. the contents of those subfolders are not listed
1. files are listed in alphabetical order
1. file names can have a mix of upper and lower case characters

Chaining together directories and subdirectories is called creating a path.  In the previous example '/home/jovyan/' is the path. Since LinuxTerminal is a subdirectory, we can append it to the path.  For example:

In [3]:
ls -F /home/jovyan/LinuxTerminal/

Linux.Terminal.Setup.ipynb


In this case, there is only one file in the LinuxTerminal subdirectory and there are no further subdirectories.  We have reached the end of the path.

## Files

### File contents

Just like `ls` displays the contents of a subdirectory, `cat` displays the contents of a file ... with some interpretation.  For example:

In [4]:
cat /etc/debian_version

buster/sid


`cat` displays the characters that are in the file. ( Notice the absence of a period at the end. )  However ...

What is really on the storage device is actually just a series of 1's and 0's.  And we can display that using the `xxd` command.  For example,

In [5]:
xxd -b -g0 /etc/debian_version | cut -d' ' -f2 | tr -d '\n' ; echo 

0110001001110101011100110111010001100101011100100010111101110011011010010110010000001010


We can modify the previous command to dispay the interpretation:

In [6]:
xxd -b -g0 /etc/debian_version | cut -d' ' -f2-

011000100111010101110011011101000110010101110010  buster
0010111101110011011010010110010000001010          /sid.


We can modify it further to format the 1's and 0's into groups of eight, called a byte, so that they are easier to view as well as prefix each line with a count ( notice that the count is zero-based. )

In [7]:
xxd -b -g1 /etc/debian_version

00000000: 01100010 01110101 01110011 01110100 01100101 01110010  buster
00000006: 00101111 01110011 01101001 01100100 00001010           /sid.


On the far right of the output, we see that the 1's and 0's get interpreted as the letters 'b', 'u', 's', etc.  Also notice that the last byte '00001010' was displayed as a '.' That byte is actually a non-printable character, one of many.  This one happens to be the end-of-line ( aka newline or '\n' ) character, which we will encounter more of later on.  Other non-printable characters frequently encountered include tab ( '\t' ) , carriage return ( '\r' ), and null ( '\0' ).

### Lines

One way to interpret a file stream is as a collection of "lines" with each line being a collection of "text" characters.  Many files use this approach, e.g. CSV, YAML, HTML.  For these "text" files, a program can read the file one character at a time until it gets to an "end-of-line" character, then it can operate on that line, then read the next line.  `cat` does this with every line in a file. For example, we can have `cat` prefix each line with the line number:

In [8]:
cat -n /etc/debian_version

     1	buster/sid


That's not very exciting with a file that has only one line.  So, here's the same command run on a file with multiple lines:

In [9]:
cat -n /etc/os-release

     1	NAME="Ubuntu"
     2	VERSION="18.04.6 LTS (Bionic Beaver)"
     3	ID=ubuntu
     4	ID_LIKE=debian
     5	PRETTY_NAME="Ubuntu 18.04.6 LTS"
     6	VERSION_ID="18.04"
     7	HOME_URL="https://www.ubuntu.com/"
     8	SUPPORT_URL="https://help.ubuntu.com/"
     9	BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
    10	PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
    11	VERSION_CODENAME=bionic
    12	UBUNTU_CODENAME=bionic


In this case, `cat` reads each character until it gets to the end of the line, prints the line number followed by the line, then repeats the process until it gets to the end of the file.

Many other commands use this pattern of reading a line, operate on it, then repeat.  Let's look at a few:
- head
- tail
- cut
- wc
- file

BTW, for all these commands, much more details on options and how they work can be found using a Google search for "unix man " followed by the command.  For example, "[unix man head](https://www.google.com/search?q=unix+man+head)"

In [10]:
# head displays the first 10 lines of a file if not given any options
## you can specify more or fewer lines by giving it the option -n X, where X is a whole number
## here we get the first 4 words from a dictionary file
head -n 4 /usr/share/dict/words

A
A'asia
A's
AATech


In [11]:
# tail displays the last 10 lines of a file if not given any options
## you can specify more or fewer lines by giving it the option -n X, where X is a whole number
## here we get the last 4 words from a dictionary file
tail -n 4 /usr/share/dict/words

évolué
évolués
événement
événements


In [12]:
# cut displays the character range specified by the -c option or a field range specified by the -f option
## range is specified using 1-based counting
## here we get the first 6 characters from the /etc/debian_version file.
cat /etc/debian_version
cut -c 1-6 /etc/debian_version

buster/sid
buster


In [13]:
## here we get characters 7-10 from the /etc/debian_version file.
cut -c 7-10 /etc/debian_version

/sid


In [15]:
# wc gives a summary of how many lines, words, and characters there are in a file
wc /usr/share/dict/words

 654749  654749 6876726 /usr/share/dict/words


In [17]:
# file gives you a reasonable guess as to what type of file it is.
file /usr/share/dict/words
file /etc/dictionaries-common/words
file /usr/share/dict/american-english-insane
file /etc/debian_version
file /bin/grep

/usr/share/dict/words: symbolic link to /etc/dictionaries-common/words
/etc/dictionaries-common/words: symbolic link to /usr/share/dict/american-english-insane
/usr/share/dict/american-english-insane: UTF-8 Unicode text
/etc/debian_version: ASCII text
/bin/grep: ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 3.2.0, BuildID[sha1]=4806f6fd2346800fffcaeedb877623aa54cf94e8, stripped


The caveat is that these programs only work if the file is organized as a "text" file.  That is, the bytes are interpreted as alpha-numeric characters with line endings.  When some other convention is used, then the file is termed a "binary" file. This can lead to some confusion as all files are "binary".  The difference is in how the bytes are organized in the file and interpreted by some program.

## Creating a file

The previous examples used pre-existing files.  Now we will use some commands that will create data and then put them into a file. We will explore the following commands:
- date
- echo
- seq
- curl

In [18]:
# date prints the date
date

Wed Oct  5 22:32:46 UTC 2022


We can tell a command to put the data into a file by redirecting its output.  That is done using the '>' symbol.  For example, to save the output from the `date` to a file called `date.txt`:

In [19]:
# show that date.txt does not exist
ls -F

data-structures.files.folders.ipynb
env.rc
README.md


In [20]:
# generate a date and redirect the output to the date.txt file
date > date.txt


In [21]:
# show that date.txt now does exist
ls -F

data-structures.files.folders.ipynb
date.txt
env.rc
README.md


In [22]:
# display the contents of the date.txt file with a line number
cat -n date.txt

     1	Wed Oct  5 22:33:19 UTC 2022


In [23]:
# echo displays the provided text
echo 'Hello, world!'

Hello, world!


In [24]:
# to save output to a file
echo 'Hello, world!' > hw.txt

In [25]:
# display the contents
cat -n hw.txt

     1	Hello, world!


In [26]:
# seq generates a range of numbers
seq 1 10 > seq.txt
cat -n seq.txt

     1	1
     2	2
     3	3
     4	4
     5	5
     6	6
     7	7
     8	8
     9	9
    10	10


In [27]:
# curl GETs a webpage
## here it downloads a file containing air quality data from the city of Albuquerque
curl -s http://data.cabq.gov/airquality/aqindex/history/042222.0017 > abq.air-quality.dat
head abq.air-quality.dat


BEGIN_FILE
FORMAT_VERSION,2
AGENCY,0017
FILENAME,042222.0017
DATA_VERSION,201904222215
TZONE,MST,7
BEGIN_GROUP
VARIABLE,CO
DATA_TYPE,POINT
MEASUREMENT_TYPE,SAMPLE


## Command pipeline

Much like one can do method chaining in Python, Ruby, JavaScript, and other languages, commands can be piped together using a vertical bar '|'.  In this way, the output of one command can be piped as input into the next command.  For example:

In [28]:
# here the first ten lines of a file are numbered
head abq.air-quality.dat | cat -n

     1	BEGIN_FILE
     2	FORMAT_VERSION,2
     3	AGENCY,0017
     4	FILENAME,042222.0017
     5	DATA_VERSION,201904222215
     6	TZONE,MST,7
     7	BEGIN_GROUP
     8	VARIABLE,CO
     9	DATA_TYPE,POINT
    10	MEASUREMENT_TYPE,SAMPLE


In [29]:
# here only the first field is displayed from the first ten lines and then numbered
head abq.air-quality.dat | cut -d, -f 1 | cat -n

     1	BEGIN_FILE
     2	FORMAT_VERSION
     3	AGENCY
     4	FILENAME
     5	DATA_VERSION
     6	TZONE
     7	BEGIN_GROUP
     8	VARIABLE
     9	DATA_TYPE
    10	MEASUREMENT_TYPE


In [31]:
cat -n abq.air-quality.dat | shuf -n 10

    73	END_DTG,201904222159
   145	START_REF,0
   169	North Valley        ,350011013,10.2,9,8.1,7,6.3,5.7,8.4,13.6,17.9,20.6,20.6,20.3,21.4,21,20.5,20.3,19.5,17.1,11.7,11.1,10,10.1
   243	STATIONS,1
   124	START_DTG,201904220000
   211	END_DATA
   202	INTERVAL,60
   196	VARIABLE,WDV
   216	MEASUREMENT_TYPE,SAMPLE
    90	END_GROUP


In [33]:
shuf -n 100 /usr/share/dict/words | cut -c1-10 | head | cat -n


     1	Herizes
     2	Tobago's
     3	Carnaria
     4	counterpic
     5	mommuck
     6	chippewyan
     7	pseudocand
     8	Chignik
     9	subcommend
    10	boccia


In [34]:
ls -la

total 424
drwxr-xr-x 1 jovyan jovyan   4096 Oct  5 22:41  ./
drwxr-xr-x 1 jovyan jovyan   4096 Oct  5 22:11  ../
-rw-r--r-- 1 jovyan jovyan 349196 Oct  5 22:41 '4b1 - Naive Bayes.pdf'
-rw-r--r-- 1 jovyan jovyan   8508 Oct  5 22:34  abq.air-quality.dat
-rw-r--r-- 1 jovyan jovyan  25869 Oct  5 22:38  data-structures.files.folders.ipynb
-rw-r--r-- 1 jovyan jovyan     29 Oct  5 22:33  date.txt
-rw-r--r-- 1 jovyan jovyan    324 Oct  5 21:59  env.rc
-rw-r--r-- 1 jovyan jovyan     14 Oct  5 22:33  hw.txt
drwxr-xr-x 2 jovyan jovyan   4096 Oct  5 22:06  .ipynb_checkpoints/
-rw-r--r-- 1 jovyan jovyan    231 Oct  5 21:59  README.md
-rw-r--r-- 1 jovyan jovyan     21 Oct  5 22:34  seq.txt


In [35]:
file "4b1 - Naive Bayes.pdf"

4b1 - Naive Bayes.pdf: PDF document, version 1.4
